In [262]:
import pickle
import pandas as pd
import sklearn
from sklearn.metrics import f1_score

pd.set_option("max_colwidth", 80)

In [263]:
# filename = 'psgan_seed_12_val_latest_cropped'
filename = 'pix2pix_seed_32_test_latest_cropped'


# [((target, image), dist), ..., ()]


### GENERATED -- ORIGINAL
if 'val' in filename:
    filename_generated_original = 'saved_distances/' + 'distances__datasets_1200_w40_final_images_val_cropped__' + filename + '.pkl'
elif 'test' in filename:
    filename_generated_original = 'saved_distances/' + 'distances__datasets_1200_w40_final_images_test_cropped__' + filename + '.pkl'

with open(filename_generated_original, 'rb') as f:
    gen_orig_distances = pickle.load(f)

### GENERATED -- TRAIN
filename_generated_train = 'saved_distances/' + 'distances__datasets_1200_w40_final_images_train_cropped__' + filename + '.pkl'
with open(filename_generated_train, 'rb') as f:
    gen_train_distances = pickle.load(f)
    
    
### GENERATED -- GENERATED
filename_generated_generated = 'saved_distances/' + 'distances__' + filename + '__.pkl'
with open(filename_generated_generated, 'rb') as f:
    gen_gen_distances = pickle.load(f)

### THRESHOLD

In [264]:
threshold = 0.22

## GENERATED -- ORIGINAL

In [265]:
## out of all matches, what percentage are below threshold

gen_orig_dist_df = pd.DataFrame(gen_orig_distances)
gen_orig_dist_df.head()
try:
    gen_orig_dist_df[['target', 'image']] = pd.DataFrame(gen_orig_dist_df[0].tolist(), index=gen_orig_dist_df.index)
    gen_orig_dist_df = gen_orig_dist_df.rename(columns={0: "data", 1: "dist"})
except:
    pass
gen_orig_dist_df['match'] = gen_orig_dist_df['target'] == gen_orig_dist_df['image']
# gen_orig_dist_df['rank'] = 
gen_orig_dist_df['below'] = gen_orig_dist_df['dist'] < threshold

gen_orig_dist_match_df = gen_orig_dist_df[gen_orig_dist_df['match'] == True]
gen_orig_dist_match_df.head()

,data,dist,target,image,match,below
0,"(strasbourg_000001_051574_28, strasbourg_000001_051574_28)",0.1549,strasbourg_000001_051574_28,strasbourg_000001_051574_28,True,True
5,"(zurich_000065_000019_31, zurich_000065_000019_31)",0.2080,zurich_000065_000019_31,zurich_000065_000019_31,True,True
20,"(strasbourg_000000_029915_4, strasbourg_000000_029915_4)",0.2231,strasbourg_000000_029915_4,strasbourg_000000_029915_4,True,False
30,"(hamburg_000000_039264_0, hamburg_000000_039264_0)",0.2296,hamburg_000000_039264_0,hamburg_000000_039264_0,True,False
33,"(hamburg_000000_038915_7, hamburg_000000_038915_7)",0.2305,hamburg_000000_038915_7,hamburg_000000_038915_7,True,False


#### Percent of gen-original images with dist below threshold

In [266]:
num_matches = len(gen_orig_dist_match_df)
print('num_matches')
print(num_matches)
num_matches_below_threshold = len(gen_orig_dist_match_df[gen_orig_dist_match_df['below'] == True])
print('num_matches_below_threshold')
print(num_matches_below_threshold)

percent_not_anonymized = round(num_matches_below_threshold / num_matches, 3)
print('----------------------------')
print('percent_not_anonymized')
print(percent_not_anonymized)
print('----------------------------')

display(gen_orig_dist_match_df[gen_orig_dist_match_df['below'] == True])

num_matches
429
num_matches_below_threshold
2
----------------------------
percent_not_anonymized
0.005
----------------------------


,data,dist,target,image,match,below
0,"(strasbourg_000001_051574_28, strasbourg_000001_051574_28)",0.1549,strasbourg_000001_051574_28,strasbourg_000001_051574_28,True,True
5,"(zurich_000065_000019_31, zurich_000065_000019_31)",0.2080,zurich_000065_000019_31,zurich_000065_000019_31,True,True


#### Recall @ Rank 1 

In [267]:
gen_orig_distances[0]

(('strasbourg_000001_051574_28', 'strasbourg_000001_051574_28'), 0.1549)

In [268]:
gen_orig_dist_df.sort_values(by=['dist'])
rank1_gen_orig_dist_df = gen_orig_dist_df.drop_duplicates(subset=['target'])
rank1_gen_orig_dist_df

print('num_matches')
print(num_matches)

num_rank1_matches = len(rank1_gen_orig_dist_df[rank1_gen_orig_dist_df['match'] == True])
print('num_rank1_matches')
print(num_rank1_matches)
recall_rank_1 = round(num_rank1_matches / num_matches, 3)

print('----------------------------')
print('recall@rank1')
print(recall_rank_1)
print('----------------------------')

rank1_gen_orig_dist_df[rank1_gen_orig_dist_df['match'] == True]

num_matches
429
num_rank1_matches
8
----------------------------
recall@rank1
0.019
----------------------------


,data,dist,target,image,match,below
0,"(strasbourg_000001_051574_28, strasbourg_000001_051574_28)",0.1549,strasbourg_000001_051574_28,strasbourg_000001_051574_28,True,True
20,"(strasbourg_000000_029915_4, strasbourg_000000_029915_4)",0.2231,strasbourg_000000_029915_4,strasbourg_000000_029915_4,True,False
30,"(hamburg_000000_039264_0, hamburg_000000_039264_0)",0.2296,hamburg_000000_039264_0,hamburg_000000_039264_0,True,False
33,"(hamburg_000000_038915_7, hamburg_000000_038915_7)",0.2305,hamburg_000000_038915_7,hamburg_000000_038915_7,True,False
60,"(zurich_000006_000019_9, zurich_000006_000019_9)",0.2459,zurich_000006_000019_9,zurich_000006_000019_9,True,False
114,"(aachen_000016_000019_27, aachen_000016_000019_27)",0.2570,aachen_000016_000019_27,aachen_000016_000019_27,True,False
369,"(frankfurt_000001_064130_3, frankfurt_000001_064130_3)",0.2793,frankfurt_000001_064130_3,frankfurt_000001_064130_3,True,False
694,"(zurich_000065_000019_41, zurich_000065_000019_41)",0.2916,zurich_000065_000019_41,zurich_000065_000019_41,True,False


## GENERATED -- TRAIN

In [269]:
## out of all pairs, what percentage are below threshold


gen_train_dist_df = pd.DataFrame(gen_train_distances)
gen_train_dist_df.head()
try:
    gen_train_dist_df[['target', 'image']] = pd.DataFrame(gen_train_dist_df[0].tolist(), index=gen_train_dist_df.index)
    gen_train_dist_df = gen_train_dist_df.rename(columns={0: "data", 1: "dist"})
except:
    pass
gen_train_dist_df['match'] = gen_train_dist_df['target'] == gen_train_dist_df['image']
gen_train_dist_df['below'] = gen_train_dist_df['dist'] < threshold

In [270]:
gen_train_dist_df.head()

,data,dist,target,image,match,below
0,"(cologne_000005_000019_0, hamburg_000000_053776_19)",0.1532,cologne_000005_000019_0,hamburg_000000_053776_19,False,True
1,"(stuttgart_000048_000019_7, hanover_000000_043822_14)",0.1686,stuttgart_000048_000019_7,hanover_000000_043822_14,False,True
2,"(hamburg_000000_047157_31, strasbourg_000000_018358_1)",0.1710,hamburg_000000_047157_31,strasbourg_000000_018358_1,False,True
3,"(strasbourg_000001_052297_33, hamburg_000000_053776_19)",0.1881,strasbourg_000001_052297_33,hamburg_000000_053776_19,False,True
4,"(hamburg_000000_047157_31, bremen_000261_000019_1)",0.1971,hamburg_000000_047157_31,bremen_000261_000019_1,False,True


#### Percent of gen-train images with dist below threshold

In [271]:
num_gen_images = len(gen_train_dist_df['target'].unique())
print('num_gen_images', num_gen_images)

gen_train_pairs_below_df = gen_train_dist_df[gen_train_dist_df['below'] == True]
num_gen_images_copied_from_train = len(gen_train_pairs_below_df['target'].unique())
print('num_gen_images_copied_from_train', num_gen_images_copied_from_train)


percent_gen_similar_to_train = round(num_gen_images_copied_from_train / num_gen_images, 3)
print('----------------------------')
print('percent_gen_similar_to_train')
print(percent_gen_similar_to_train)
print('----------------------------')

display(gen_train_pairs_below_df)

num_gen_images 429
num_gen_images_copied_from_train 15
----------------------------
percent_gen_similar_to_train
0.035
----------------------------


,data,dist,target,image,match,below
0,"(cologne_000005_000019_0, hamburg_000000_053776_19)",0.1532,cologne_000005_000019_0,hamburg_000000_053776_19,False,True
1,"(stuttgart_000048_000019_7, hanover_000000_043822_14)",0.1686,stuttgart_000048_000019_7,hanover_000000_043822_14,False,True
2,"(hamburg_000000_047157_31, strasbourg_000000_018358_1)",0.1710,hamburg_000000_047157_31,strasbourg_000000_018358_1,False,True
3,"(strasbourg_000001_052297_33, hamburg_000000_053776_19)",0.1881,strasbourg_000001_052297_33,hamburg_000000_053776_19,False,True
4,"(hamburg_000000_047157_31, bremen_000261_000019_1)",0.1971,hamburg_000000_047157_31,bremen_000261_000019_1,False,True
5,"(munster_000167_000019_0, frankfurt_000000_020215_5)",0.2019,munster_000167_000019_0,frankfurt_000000_020215_5,False,True
6,"(hamburg_000000_038915_10, frankfurt_000001_012870_6)",0.2045,hamburg_000000_038915_10,frankfurt_000001_012870_6,False,True
7,"(erfurt_000072_000019_4, weimar_000087_000019_8)",0.2074,erfurt_000072_000019_4,weimar_000087_000019_8,False,True
8,"(erfurt_000035_000019_15, hamburg_000000_047057_36)",0.2082,erfurt_000035_000019_15,hamburg_000000_047057_36,False,True
9,"(hamburg_000000_047157_31, dusseldorf_000015_000019_0)",0.2102,hamburg_000000_047157_31,dusseldorf_000015_000019_0,False,True


## GENERATED -- GENERATED

In [272]:
# ## out of all gen pairs, what percentage are below threshold (ignore matches)

gen_gen_dist_df = pd.DataFrame(gen_gen_distances)
gen_gen_dist_df.head()
try:
    gen_gen_dist_df[['target', 'image']] = pd.DataFrame(gen_gen_dist_df[0].tolist(), index=gen_gen_dist_df.index)
    gen_gen_dist_df = gen_gen_dist_df.rename(columns={0: "data", 1: "dist"})
except:
    pass
gen_gen_dist_df['match'] = gen_gen_dist_df['target'] == gen_gen_dist_df['image']
gen_gen_dist_df['below'] = gen_gen_dist_df['dist'] < threshold
gen_gen_dist_nonmatch_df = gen_gen_dist_df[gen_gen_dist_df['match'] == False]
gen_gen_dist_nonmatch_df.head()

,data,dist,target,image,match,below
429,"(stuttgart_000130_000019_3, munster_000144_000019_6)",0.1165,stuttgart_000130_000019_3,munster_000144_000019_6,False,True
430,"(cologne_000023_000019_17, frankfurt_000001_062793_13)",0.1173,cologne_000023_000019_17,frankfurt_000001_062793_13,False,True
431,"(bremen_000055_000019_1, hamburg_000000_105296_30)",0.1220,bremen_000055_000019_1,hamburg_000000_105296_30,False,True
432,"(bremen_000055_000019_1, jena_000084_000019_7)",0.1287,bremen_000055_000019_1,jena_000084_000019_7,False,True
433,"(stuttgart_000134_000019_3, hamburg_000000_096063_10)",0.1288,stuttgart_000134_000019_3,hamburg_000000_096063_10,False,True


In [273]:
num_gen_images = len(gen_gen_dist_df['target'].unique())
print('num_gen_images', num_gen_images)

gen_gen_pairs_below_df = gen_gen_dist_nonmatch_df[gen_gen_dist_nonmatch_df['below'] == True]
num_gen_images_with_shared_identity = len(gen_gen_pairs_below_df['target'].unique())
print('num_gen_images_with_shared_identity', num_gen_images_with_shared_identity)

percent_gen_with_shared_identity = round(num_gen_images_with_shared_identity / num_gen_images, 3)
print('----------------------------')
print('percent_gen_with_shared_identity')
print(percent_gen_with_shared_identity)
print('----------------------------')

display(gen_gen_pairs_below_df.head())

num_gen_images 429
num_gen_images_with_shared_identity 242
----------------------------
percent_gen_with_shared_identity
0.564
----------------------------


,data,dist,target,image,match,below
429,"(stuttgart_000130_000019_3, munster_000144_000019_6)",0.1165,stuttgart_000130_000019_3,munster_000144_000019_6,False,True
430,"(cologne_000023_000019_17, frankfurt_000001_062793_13)",0.1173,cologne_000023_000019_17,frankfurt_000001_062793_13,False,True
431,"(bremen_000055_000019_1, hamburg_000000_105296_30)",0.1220,bremen_000055_000019_1,hamburg_000000_105296_30,False,True
432,"(bremen_000055_000019_1, jena_000084_000019_7)",0.1287,bremen_000055_000019_1,jena_000084_000019_7,False,True
433,"(stuttgart_000134_000019_3, hamburg_000000_096063_10)",0.1288,stuttgart_000134_000019_3,hamburg_000000_096063_10,False,True


### RESULTS

In [274]:
print('percent_not_anonymized')
print('recall@rank1')
print('percent_gen_similar_to_train')
print('percent_gen_with_shared_identity')
print()

print(percent_not_anonymized)
print(recall_rank_1)
print(percent_gen_similar_to_train)
print(percent_gen_with_shared_identity)

percent_not_anonymized
recall@rank1
percent_gen_similar_to_train
percent_gen_with_shared_identity

0.005
0.019
0.035
0.564


In [275]:
print(percent_not_anonymized, recall_rank_1, percent_gen_similar_to_train, percent_gen_with_shared_identity)

0.005 0.019 0.035 0.564


In [276]:
pd.DataFrame([percent_not_anonymized, recall_rank_1, percent_gen_similar_to_train, percent_gen_with_shared_identity], index=['perc_not_anonymized', 'recall_rank_1', 'perc_gen_similar_to_train', 'perc_gen_with_shared_identity'], columns=['A']).T


,perc_not_anonymized,recall_rank_1,perc_gen_similar_to_train,perc_gen_with_shared_identity
A,0.005,0.019,0.035,0.564
